In [1]:
import pandas as pd
import guidance
import openai
from sklearn.metrics import accuracy_score
import random
from tqdm import tqdm
from collections import defaultdict
import time
openai.api_key = ""


In [3]:
df = pd.read_csv("../data/data.csv")
df

,input_text,output_text
0,How much did my fleet idle last month?,Idling-Idling duration
1,how many Kilometers the vehicle name 1943 driv...,Asset Utilization-vehicle distance
2,how many Kilometers the vehicle ID b412 drive ...,Asset Utilization-vehicle distance
3,how many Kilometers the vehicle ID b412 drive ...,Trips-distance
4,What is the size of the MiniBulk Fleet?,Asset Monitoring and Groups-Company groups and...
...,...,...
294,which of my vehicles is ready to be replaced w...,Sustainability-EV suitability assessment (EVSA)
295,which of my vehicles has the most speeding vio...,"Safety-Safety exceptions including speeding, h..."
296,how much of that idle is in the state of illin...,Idling-Idling location/exception
297,how many vehicles are in our system,Asset Monitoring and Groups-active vehicles


In [4]:
questions = df.input_text.tolist()
categories = df.output_text.tolist()
options = list(set(categories))
len(categories)

299

In [5]:
df.input_text.tolist()[9]

'What is the top 10 of the most used trucks in terms of distance and hours driven in the Group "Ontario" from \'2023-01-01\' to \'2023-06-30\' ?'

In [6]:
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo-instruct")

In [7]:
instruction_prompt = guidance("""Below is an instruction that describes a task, paired with an input that provides further context. Write a response
that appropriately completes the request.
### Instruction:
{{ prompt }}
### Input:
{{ question }}
### Response:
{{gen 'category_guidance' temperature=0}}""",
silent=True,
async_mode=False,
stream=False,
caching=False,
log=True)

In [8]:
envolve_prompt = guidance("""Query:
Please follow the instruction step-by-step to generate a better prompt.
1. Identify the different parts between the Prompt 1 and Prompt 2:
Prompt 1: {{ prompt1 }}
Prompt 2: {{ prompt2 }}
2. Randomly mutate the different parts
3. Combine the different parts with Prompt 3, selectively replace it with the different parts in Step 2 and generate a new prompt.
Prompt 3: {{ current_best_prompt }}
4. Cross over the prompt in the Step 3 with the following basic prompt and generate a final prompt.
Prompt final: {{gen 'final_prompt' temperature=1}}""",
silent=True,
async_mode=False,
stream=False,
caching=False,
log=True)

In [9]:
initial_prompts = ["""Examine the given text snippet closely and classify it into one of the following categories based on the context and content: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', or 'Sustainability'.""",
                  """After analyzing the provided text, determine the most appropriate category it belongs to. The possible categories are: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', and 'Sustainability'.""",
                  """Please perform Sentiment Classification task. Given the sentence, assign a sentiment label from ['Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', 'Sustainability']. Return label only without any other text.""",
                   """In this task, you are given sentences from customer questions. Based on the given question, classify it to one of the five classes: (1) Idling, (2) Asset Utilization, (3) Asset Monitoring and Groups, (4) Fuel and Energy Consumption, (5) Engine Faults, (6) Device Health, and (7) Sustainability.""",
                   """Evaluate the question the customers have according to the purposes summary they have been given, using phrases such as "Idling", "Asset Utilization", "Asset Monitoring and Groups", "Fuel and Energy Consumption", "Engine Faults", " Device Health" or "Sustainability"."""
                  ]


In [10]:
envolve_program = envolve_prompt(
                prompt1=initial_prompts[0],
                prompt2=initial_prompts[1],
    current_best_prompt=initial_prompts[3],
)

In [11]:
print(initial_prompts[0])

Examine the given text snippet closely and classify it into one of the following categories based on the context and content: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', or 'Sustainability'.


In [12]:
print(initial_prompts[1])

After analyzing the provided text, determine the most appropriate category it belongs to. The possible categories are: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', and 'Sustainability'.


In [13]:
print(initial_prompts[3])

In this task, you are given sentences from customer questions. Based on the given question, classify it to one of the five classes: (1) Idling, (2) Asset Utilization, (3) Asset Monitoring and Groups, (4) Fuel and Energy Consumption, (5) Engine Faults, (6) Device Health, and (7) Sustainability.


In [14]:
print(envolve_program["final_prompt"].strip())

Categorize the following customer question into one of the provided classes: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', or 'Sustainability'.


In [15]:
index = 10
instruction_program = instruction_prompt(
                prompt=initial_prompts[0],
                question=questions[index], options=options)
cat_pred = instruction_program["category_guidance"].strip()
for cat in options:
    if cat in cat_pred:
        print("pred:" ,cat)
print("real:", categories[index])

real: Trips-group


# evaluation function

In [15]:
def evaluation(prompt, sample=10, seed=42):
    if seed is not None:
        random.seed(seed)
        
    sample_idx = random.sample(list(range(len(questions))), sample)
    test_questions = [questions[i] for i in sample_idx]
    test_categories = [categories[i] for i in sample_idx]
    preds = []
    added_pred = False
    
    for question in tqdm(test_questions):
        executed_program = instruction_prompt(
                prompt=prompt,
                question=question)
        gen = executed_program["category_guidance"].strip()
        
        for cat in options:
            if cat in gen:
                preds.append(cat)
                added_pred = True
                break  # break out of the inner loop once a category is found
                
        if not added_pred:
            preds.append("")
            
        added_pred = False
        time.sleep(1)  # Pauses a bit since opanai api call might pendding

        
    return accuracy_score(test_categories, preds), test_categories, preds


In [16]:
score, test_categories, preds = evaluation(initial_prompts[0])

100%|██████████| 10/10 [00:20<00:00,  2.04s/it]


In [17]:
score

0.9

In [18]:
test_categories

['Fuel and Energy Consumption',
 'Asset Utilization',
 'Asset Utilization',
 'Asset Utilization',
 'Asset Utilization',
 'Fuel and Energy Consumption',
 'Engine Faults',
 'Device Health',
 'Idling',
 'Fuel and Energy Consumption']

In [19]:
preds

['Fuel and Energy Consumption',
 'Asset Utilization',
 'Asset Utilization',
 'Asset Utilization',
 'Asset Utilization',
 'Fuel and Energy Consumption',
 'Engine Faults',
 'Asset Monitoring and Groups',
 'Idling',
 'Fuel and Energy Consumption']

# Envolve steps

In [20]:
print(initial_prompts)

["Examine the given text snippet closely and classify it into one of the following categories based on the context and content: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', or 'Sustainability'.", "After analyzing the provided text, determine the most appropriate category it belongs to. The possible categories are: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', and 'Sustainability'.", "Please perform Sentiment Classification task. Given the sentence, assign a sentiment label from ['Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', 'Sustainability']. Return label only without any other text.", 'In this task, you are given sentences from customer questions. Based on the given question, classify it to one of the five classes: (1) Idling, (2) Asset Utilizat

In [21]:
iterations = 10
new_gen_each_round = 3
population = initial_prompts.copy()
keep = 5
eval_sample = 100

In [22]:
all_generations = defaultdict(list)

for iter in tqdm(range(iterations)):
    current_scores = []
    print(f"Generation: {iter}")
    for prompt in population:
        score, test_categories, preds = evaluation(prompt, sample=eval_sample, seed=42) # seed=iter
        all_generations[iter].append((prompt, score))
        current_scores.append((prompt, score))
    current_scores = sorted(current_scores, key=lambda x: x[1], reverse=True)[:keep]
    best_ind = current_scores[0][0]
    
    # envolve
    for _ in range(new_gen_each_round):
        sampled_inds = [p for p, s in random.sample(current_scores[1:], 2)]
        envolve_program = envolve_prompt(
                        prompt1=sampled_inds[0],
                        prompt2=sampled_inds[1],
            current_best_prompt=best_ind,
        )
        new_gen = envolve_program["final_prompt"].strip()
        score, test_categories, preds = evaluation(new_gen, sample=eval_sample, seed=42) # seed=iter
        all_generations[iter].append((new_gen, score))
        current_scores.append((new_gen, score))
        
    current_scores = sorted(current_scores, key=lambda x: x[1], reverse=True)[:keep]
    population = [p for p, s in current_scores]
    print(f"current best: {current_scores[0][1]}, : {current_scores[0][0]}")
    
    
        

  0%|          | 0/10 [00:00<?, ?it/s]

Generation: 0



100%|██████████| 100/100 [02:46<00:00,  1.67s/it]

100%|██████████| 100/100 [02:30<00:00,  1.50s/it]

100%|██████████| 100/100 [02:13<00:00,  1.33s/it]

100%|██████████| 100/100 [02:36<00:00,  1.56s/it]

100%|██████████| 100/100 [03:00<00:00,  1.80s/it]

100%|██████████| 100/100 [03:21<00:00,  2.01s/it]

100%|██████████| 100/100 [03:27<00:00,  2.08s/it]

 10%|█         | 1/10 [22:30<3:22:36, 1350.75s/it]

current best: 0.85, : After analyzing the provided text, determine the most appropriate category it belongs to. The possible categories are: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', and 'Sustainability'.
Generation: 1



100%|██████████| 100/100 [02:31<00:00,  1.52s/it]

100%|██████████| 100/100 [02:29<00:00,  1.49s/it]

100%|██████████| 100/100 [02:35<00:00,  1.56s/it]

100%|██████████| 100/100 [02:13<00:00,  1.33s/it]

100%|██████████| 100/100 [02:45<00:00,  1.65s/it]

100%|██████████| 100/100 [02:14<00:00,  1.34s/it]

100%|██████████| 100/100 [02:25<00:00,  1.46s/it]

 20%|██        | 2/10 [42:02<2:46:02, 1245.35s/it]

...

current best: 0.86, : Please use the following instructions to properly classify the given text and return the most suitable category: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', or 'Sustainability'. After following these steps, please provide the relevant label for the text.
Generation: 8



100%|██████████| 100/100 [02:24<00:00,  1.45s/it]

100%|██████████| 100/100 [02:20<00:00,  1.40s/it]

100%|██████████| 100/100 [02:27<00:00,  1.48s/it]

100%|██████████| 100/100 [02:31<00:00,  1.52s/it]

100%|██████████| 100/100 [02:34<00:00,  1.55s/it]

100%|██████████| 100/100 [02:30<00:00,  1.50s/it]

100%|██████████| 100/100 [03:36<00:00,  2.16s/it]

 90%|█████████ | 9/10 [3:00:29<20:15, 1215.09s/it]

current best: 0.86, : Please use the following instructions to properly classify the given text and return the most suitable category: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', or 'Sustainability'. After following these steps, please provide the relevant label for the text.
Generation: 9



100%|██████████| 100/100 [02:23<00:00,  1.43s/it]

100%|██████████| 100/100 [02:19<00:00,  1.40s/it]

100%|██████████| 100/100 [02:27<00:00,  1.47s/it]

100%|██████████| 100/100 [02:33<00:00,  1.53s/it]

100%|██████████| 100/100 [02:33<00:00,  1.53s/it]

100%|██████████| 100/100 [02:34<00:00,  1.54s/it]

100%|██████████| 100/100 [02:27<00:00,  1.47s/it]

100%|██████████| 10/10 [3:20:29<00:00, 1202.95s/it]

current best: 0.88, : Please correctly identify the suitable category for the given text by following these instructions: 'Idling', (2) Asset Utilization, (3) Asset Monitoring and Groups, (4) Fuel and Energy Consumption, (5) Engine Faults, (6) Device Health, or (7) Sustainability. After thoroughly analyzing the text, kindly provide the appropriate label.


In [23]:
all_generations.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [24]:
all_generations[0]

[("Examine the given text snippet closely and classify it into one of the following categories based on the context and content: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', or 'Sustainability'.",
  0.77),
 ("After analyzing the provided text, determine the most appropriate category it belongs to. The possible categories are: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', and 'Sustainability'.",
  0.85),
 ("Please perform Sentiment Classification task. Given the sentence, assign a sentiment label from ['Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', 'Sustainability']. Return label only without any other text.",
  0.79),
 ('In this task, you are given sentences from customer questions. Based on the given question, classify it to one of the five class

In [25]:
best_s = 0
best_p = ""
for i, data in all_generations.items():
    for p, s in data:
        print(s)
        print(p)
        print("#"*10)
        print()
        if s > best_s:
            best_s = s
            best_p = p

0.77
Examine the given text snippet closely and classify it into one of the following categories based on the context and content: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', or 'Sustainability'.
##########

0.85
After analyzing the provided text, determine the most appropriate category it belongs to. The possible categories are: 'Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', and 'Sustainability'.
##########

0.79
Please perform Sentiment Classification task. Given the sentence, assign a sentiment label from ['Idling', 'Asset Utilization', 'Asset Monitoring and Groups', 'Fuel and Energy Consumption', 'Engine Faults', 'Device Health', 'Sustainability']. Return label only without any other text.
##########

0.8
In this task, you are given sentences from customer questions. Based on the given question, classify it to one of the 

In [26]:
best_s

0.88

In [27]:
print(best_p)

Please correctly identify the suitable category for the given text by following these instructions: 'Idling', (2) Asset Utilization, (3) Asset Monitoring and Groups, (4) Fuel and Energy Consumption, (5) Engine Faults, (6) Device Health, or (7) Sustainability. After thoroughly analyzing the text, kindly provide the appropriate label.
